In [1]:
!pip install --upgrade cryptography
!pip install --upgrade paramiko

In [2]:
!pip install delta-spark

In [3]:
import time
from pyspark.sql import functions as F
from pyspark.sql.functions import *
from datetime import *
from pyspark.sql.window import *
from pyspark.sql import Window
from pyspark.sql.types import *
from delta import DeltaTable
from dateutil.relativedelta import relativedelta
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import *
from io import StringIO
from email.mime.application import MIMEApplication
import shutil
import os
# import oracledb
import math
import calendar
import re
import pandas as pd
from pandas import ExcelWriter
# import openpyxl 
# from openpyxl import Workbook
# from openpyxl.styles import PatternFill, Border, Side
from email.mime.base import MIMEBase
from email import encoders
import concurrent.futures
import zipfile
import shutil
from tempfile import TemporaryDirectory
from datetime import *
from pytz import timezone
import logging
import json
import requests
import tarfile
# !pip install paramiko
# import paramiko
log_information=[]

In [4]:
def append_dataframes_with_missing_columns(base_df, data_df):
    all_columns = set(base_df.columns) | set(data_df.columns) 
    for col_name in all_columns:
        if col_name.upper() not in data_df.columns and col_name.lower() not in data_df.columns:
            data_df = data_df.withColumn(col_name.upper(), lit(None).cast(base_df.schema[col_name].dataType))
         
    data_df = data_df.select([col(c.name.upper()).cast(c.dataType) for c in base_df.schema])
    appended_df = base_df.union(data_df)
    return appended_df

In [5]:
def union_dataframes_with_missing_columns(base_df, data_df):
    all_columns = set(base_df.columns) | set(data_df.columns)
    for col_name in all_columns:
        if (col_name not in base_df.columns) and (col_name.upper() not in base_df.columns) and (col_name.lower() not in base_df.columns):
            base_df = base_df.withColumn(col_name.upper(), lit(None).cast(data_df.schema[col_name].dataType))
        if (col_name not in data_df.columns) and (col_name.upper() not in data_df.columns) and (col_name.lower() not in data_df.columns):
            data_df = data_df.withColumn(col_name.upper(), lit(None).cast(base_df.schema[col_name].dataType))
    data_df = data_df.select([c.name for c in base_df.schema])
    base_df = base_df.select(data_df.columns)
    appended_df = base_df.unionByName(data_df)
    return appended_df

In [6]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.functions import col, lit, when, udf
from pyspark import SparkContext
from pyspark.ml import Estimator, Model
from pyspark.ml.feature import VectorAssembler, StringIndexer
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType, StringType
from pyspark.sql import SQLContext
from pyspark.sql import DataFrame

In [7]:
import findspark
findspark.init()  

import pyspark
import pandas as pd
from pyspark.sql import SparkSession

In [8]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit

# Initialize SparkSession
spark = SparkSession.builder.appName("AppendMissingColumns").getOrCreate()

# Create the base DataFrame with 10 columns
base_data = [
    {"id": 1, "name": "Alice", "age": 25, "city": "Mumbai", "country": "India", "gender": "Female", "income": 50000, "status": "Single", "score": 85, "category": "A"},
    {"id": 2, "name": "Bob", "age": 30, "city": "Delhi", "country": "India", "gender": "Male", "income": 60000, "status": "Married", "score": 90, "category": "B"},
    {"id": 3, "name": "Charlie", "age": 28, "city": "Bangalore", "country": "India", "gender": "Male", "income": 45000, "status": "Single", "score": 75, "category": "C"},
    {"id": 4, "name": "Diana", "age": 24, "city": "Chennai", "country": "India", "gender": "Female", "income": 70000, "status": "Single", "score": 88, "category": "A"},
    {"id": 5, "name": "Eve", "age": 29, "city": "Kolkata", "country": "India", "gender": "Female", "income": 80000, "status": "Married", "score": 92, "category": "B"}
]

base_df = spark.createDataFrame(base_data)

# Create the secondary DataFrame with fewer columns
data_data = [
    {"id": 6, "name": "Frank", "city": "Pune", "gender": "Male"},
    {"id": 7, "name": "Grace", "city": "Hyderabad", "gender": "Female"},
    {"id": 8, "name": "Henry", "city": "Jaipur", "gender": "Male"},
    {"id": 9, "name": "Ivy", "city": "Lucknow", "gender": "Female"},
    {"id": 10, "name": "Jack", "city": "Ahmedabad", "gender": "Male"}
]

data_df = spark.createDataFrame(data_data)

# Show the two DataFrames for clarity
print("Base DataFrame:")
base_df.show()

print("Data DataFrame:")
data_df.show()

result_df = append_dataframes_with_missing_columns(base_df, data_df)

print("Resulting DataFrame After Appending:")
result_df.show()


Base DataFrame:
+---+--------+---------+-------+------+---+------+-------+-----+-------+
|age|category|     city|country|gender| id|income|   name|score| status|
+---+--------+---------+-------+------+---+------+-------+-----+-------+
| 25|       A|   Mumbai|  India|Female|  1| 50000|  Alice|   85| Single|
| 30|       B|    Delhi|  India|  Male|  2| 60000|    Bob|   90|Married|
| 28|       C|Bangalore|  India|  Male|  3| 45000|Charlie|   75| Single|
| 24|       A|  Chennai|  India|Female|  4| 70000|  Diana|   88| Single|
| 29|       B|  Kolkata|  India|Female|  5| 80000|    Eve|   92|Married|
+---+--------+---------+-------+------+---+------+-------+-----+-------+

Data DataFrame:
+---------+------+---+-----+
|     city|gender| id| name|
+---------+------+---+-----+
|     Pune|  Male|  6|Frank|
|Hyderabad|Female|  7|Grace|
|   Jaipur|  Male|  8|Henry|
|  Lucknow|Female|  9|  Ivy|
|Ahmedabad|  Male| 10| Jack|
+---------+------+---+-----+

Resulting DataFrame After Appending:
+----+-----